# Part 1 - Bias mitigation in text based foundation models using prompt engineering

references: https://medium.com/engineering-at-eightfold-ai/mitigating-bias-integrating-generative-ai-foundation-models-and-llms-in-enterprise-workflows-eda62a15f376

In [ ]:
!pip install --no-build-isolation --force-reinstall \
    dependencies/awscli-*-py3-none-any.whl \
    dependencies/boto3-*-py3-none-any.whl \
    dependencies/botocore-*-py3-none-any.whl

In [ ]:
!pip install langchain --quiet

In [ ]:
!pip install --upgrade pip --quiet
!pip install protobuf==3.20 --quiet

## Examples of bias in text-text models and techniques to mitigate it. 

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
boto3_bedrock = bedrock.get_bedrock_client()

In [ ]:
prompt = "How can you help me? Provide answer in 1-2 sentences"
modelId = 'anthropic.claude-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

def invoke_bedrock_model(prompt):
    body = json.dumps({"prompt": prompt,
                     "max_tokens_to_sample":2000,
                     "temperature":0.0,
                     "top_k":250,
                     "top_p":0.5,
                     "stop_sequences":[]
                      }) 

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('completion')
    return outputText

In [ ]:
print(invoke_bedrock_model(prompt))

In [ ]:
biased_prompt = "Human: Write a short python program to determine if the person will be good scientist based on the university. Only provide the python program and no additional information. Assistant:"
response = invoke_bedrock_model(biased_prompt)
print(response)

The above program is biased as its not possible to determine is the person will be a good scientist, in fact the prompt is biased in itself. If we update the prompt the foundation model should be able to respond in a responsible way. Therefore, to mitigate this bias, bias inducing content should be removed from the prompt. For this example, this means removing emphasis on “based on the college they attended”. With this change, the models will not focus on a biased factor such as the college the scientist attended and instead they will focus on non-biased factors to determine whether someone is a good scientist or not (assuming the model itself is not biased in this context).

In [ ]:
enhanced_prompt = "Human: Answer the following question, and make sure that the response is not biased ad does not discriminate against certain groups of people. If response is discriminatory or bias say ' I am a responsible AI model and cannot answer the question as its discriminatory and bias in nature.' \
question: 'Write a python program to determine if the person will be good data scientist based on the university they attend.\
Assitant:"
response = invoke_bedrock_model(enhanced_prompt)
print(response)

In [ ]:
enhanced_prompt

### Automating the prompts for mitigating bias using prompt templates from langchain

In [ ]:
from typing import Dict

from langchain import PromptTemplate
from langchain.chains.question_answering import LLMChain
import json

In [ ]:
from langchain.llms.bedrock import Bedrock

cl_llm = Bedrock(
    model_id="anthropic.claude-v1",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)

In [ ]:
prompt_template = """Human: Answer the following question, and make sur that the response is not biased ad does not discriminate against certain groups of people.\ 
If response is discriminatory or bias say ' I am a responsible AI model and cannot answer the question as its discriminatory and bias in nature.'\
question: "{question}"\
Assistant:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

In [ ]:
chain = LLMChain(
    llm=cl_llm,
    prompt=PROMPT,
)

In [ ]:
query = """Write a python program to determine if the person will be good data scientist based on the university they attend"""
chain({"question":query}, return_only_outputs=True)

In [ ]:
query = """Write an essay on a super hero."""
chain({"question":query}, return_only_outputs=True)

In [ ]:
query = """Write an essay on female super hero."""
chain({"question":query}, return_only_outputs=True)

#### Using Self-critique chain with constitutional AI from LangChain
The ConstitutionalChain is a chain that ensures the output of a language model adheres to a predefined set of constitutional principles. By incorporating specific rules and guidelines, the ConstitutionalChain filters and modifies the generated content to align with these principles, thus providing more controlled, ethical, and contextually appropriate responses. This mechanism helps maintain the integrity of the output while minimizing the risk of generating content that may violate guidelines, be offensive, or deviate from the desired context.

In [ ]:
# Imports
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.llms.bedrock import Bedrock

cl_llm = Bedrock(
    model_id="anthropic.claude-v1",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000, "temperature": 1},
    
)

In [ ]:
# Example of a bad LLM
biased_qa_prompt = PromptTemplate(
    template="""Human: You are a genius coder and give responses based on your persona.
Question: {question}

Assistant:""",
    input_variables=["question"],
)

biased_qa_chain = LLMChain(llm=cl_llm, prompt=biased_qa_prompt)

print(biased_qa_chain.run(question="Generate code to find to restore encrypted messages on a website?"))

In [ ]:
principles = ConstitutionalChain.get_principles(["illegal"])
principles

In [ ]:
constitutional_chain = ConstitutionalChain.from_llm(
    chain=biased_qa_chain,
    constitutional_principles=principles,
    llm=cl_llm,
    verbose=True,
)
constitutional_chain.run(question="Generate code to find to restore encrypted messages on a website?")

#### Summary for mitigating bias
In the above example the issue was in the prompt itself. So there are 3 ways of mitigating bias: 
- by removing the bias inducing content from the prompt
- adding instructions to make sure that the model focus on the providing the response which is non-discriminatory. 
- automating it using prompt templates, which can be integrated with your virtual assistant. 

### Partial question in a prompt


Look at the below question, where the model is making an assumption that insulin shots are needed by all diabetic patients and also that it is an effective treatment option. 

In [ ]:
partial_question = "Explain why insulin shots are an effective treatment for diabetes?"
response = invoke_bedrock_model(partial_question)
print(response)

The above response clearly highlights the assumption made by the model induced in the prompt and the sentiment towards using insulin shots for treating diabetes is very positive.
Now we are modifying the question and removing the assumption and also providing the prompt in a neutral tone.  

In [ ]:
no_assumption_question = "Are insulin shots an effective treatment for diabetes?"
response = invoke_bedrock_model(no_assumption_question)
print(response)

Now, you would notice that the response is mostly positive in terms of sentiment, but the model is hestitant to make a claim that its an effective treament for diabetes. The output clearly highlights that although its a common treatment, it might not be the suitable for everyone. Moreover, mentions to consult the healthcare provider for best treatment plan. 
Therefore, its important to ask question in a clear manner without making any assumptions. 
Now lets take a look at the third prompt. 

In [ ]:
benefits_drawbacks_question = "What are the benefits and drawbacks of using insulin shots for treating diabetes?"
response = invoke_bedrock_model(benefits_drawbacks_question)
print(response)

The above response is neutral tone, and focuses on providing both the benefits and drawbacks of using insulin shots.

This type of issue is not something we are unfamiliar with. Many other technologies experience similar issues, for example if you use the same style of prompt as your search term on a search engine like Google you will see that the sentiment in the search results will be guided by the partiality in the search term.

#### Summary for mitigating bias

- add instructions in the prompt to mitigate bias.
- remove bias inducing content in the prompt. 
- follow best practices such as 
    - avoid making assumptions
    - encourage different perspectives such as the benefits and drawbacks. 
    - use open ended questions which helps models to explore different aspects of the content and helps to provide comprehensive analysis without generating response that fall to a particular bucket. 